In [64]:
from __future__ import print_function
import tensorflow
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io
import pandas as pd

filename = "https://drive.google.com/uc?export=download&id=1Hn-n3cigmUA-IXJykd0wYHtAXT2cigVe"
raw_text = pd.read_csv(filename)
raw_text = raw_text.iloc[:,1].values
raw_text = raw_text[:int(len(raw_text)/100)]
raw_text = " ".join(raw_text)
text = raw_text.lower()


chars = sorted(list(set(text)))
print('total chars:', len(chars))
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

maxlen = 40
step = 3
sentences = []
next_chars = []
for i in range(0, len(text) - maxlen, step):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + maxlen])
print('nb sequences:', len(sentences))

print('Vectorization...')
x = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        x[i, t, char_indices[char]] = 1
    y[i, char_indices[next_chars[i]]] = 1


print('Build model...')
model = Sequential()
model.add(LSTM(256, input_shape=(maxlen, len(chars))))
model.add(Dense(len(chars), activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

model.fit(x, y,
          batch_size=128,
          epochs=2)

json_string = model.to_json()
model.save_weights('model_weights.h5')

total chars: 64
nb sequences: 71836
Vectorization...
Build model...
Epoch 1/2
71836/71836 [==============================] - 67s 936us/step - loss: 2.7724
Epoch 2/2
71836/71836 [==============================] - 65s 911us/step - loss: 2.3595


In [0]:
def generate_joke():
  start_index = random.randint(0, len(text) - maxlen - 1)
  for diversity in [0.5]:
      #print('----- diversity:', diversity)

      generated = ''
      sentence = text[start_index: start_index + maxlen]
      generated += sentence
      #print('----- Generating with seed: "' + sentence + '"')

      for i in range(800):
          x_pred = np.zeros((1, maxlen, len(chars)))
          for t, char in enumerate(sentence):
              x_pred[0, t, char_indices[char]] = 1.

          preds = model.predict(x_pred, verbose=0)[0]
          next_index = sample(preds, diversity)
          next_char = indices_char[next_index]

          generated += next_char
          sentence = sentence[1:] + next_char

      print(sentence)

In [0]:
from keras.models import model_from_json

model = model_from_json(json_string)
model.load_weights('model_weights.h5')
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [69]:
num_of_jokes = int(input("How many jokes do you want?: "))

for i in range(num_of_jokes):
  generate_joke()

How many jokes do you want?: 10
 dovat shill now what the lois cand a th
o whe par of f ande whibe fore the pitha
ather mameras you be is the cof the i fe
 is lise in a fane sis dad dith a deint 
e wing to the bout the mexen the sard th
dor wha lithe our the inep sey the jome 
ickter on and in mon fry the wher the an
 is whit the ping bos the in what san ch
the dere is and the doring oo hat the pi
e tha way hou in the pererand ipe wand t
